# Lecture 09

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!unzip -u ../../lec/lec06/data/imdb_perf_lecture.zip -d ../../lec/lec06/data/.

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_perf_lecture'
!psql -h localhost -c 'CREATE DATABASE imdb_perf_lecture' 
!psql -h localhost -d imdb_perf_lecture -f ../../lec/lec06/data/imdb_perf_lecture.sql

# Structural Transformation: From Relations to Matrices and Back
- Matrix $\rightarrow$ Relational works.
- Relational $\rightarrow$ Matrix sometimes works!
- But how?

To start, let's take our matrix in `mm.txt`, and load it into Pandas.

In [ ]:
import pandas as pd

In [ ]:
mm = pd.read_csv('data/mm.txt', header=0)
mm

## What does an unpivot look like (Matrix -> Relational)?

In [ ]:
mm_melted = mm.melt(id_vars=['Year'])
mm_melted

Thanks to the `id_var` parameter, the `Year` column is named and repeated for all other (variable=column name, value=value) elements in the row.

In [ ]:
mm_melted[mm_melted['Year'] == 2002]

## PIVOT(UNPIVOT) = ??

In [ ]:
#mm_melted.pivot(index='variable', columns='Year')
mm_melted.pivot(index='Year', columns='variable')

In [ ]:
mm_melted.dtypes

### Extra Columns
Let's go back to `mmp.txt`. 
- Matrix or relation? 
- Try doing some PIVOT/UNPIVOT work on this.

In [ ]:
mmp = pd.read_csv('data/mmp.txt', header=0)
mmp

In [ ]:
# Unpivot
mmp_melted = mmp.melt(id_vars=['Location', 'Station', 'Year'])
mmp_melted

In [ ]:
# Repivot the unpivot

mmp_tt = mmp_melted.pivot(index='Year', columns='variable')
# mmp_tt = mmp_melted.pivot(index=['Location', 'Station', 'Year'], columns='variable')
mmp_tt

In [ ]:
mmp_tt.reset_index()

# [Extra] Multisets to Sets

### Set up connections and schema

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/postgres
import pandas as pd

In [ ]:
%%sql
DROP TABLE IF EXISTS blue;
DROP TABLE IF EXISTS red;
CREATE TABLE blue (last TEXT, first TEXT);
CREATE TABLE red (last TEXT, first TEXT);

INSERT INTO blue VALUES ('Wang', 'Daisy');
INSERT INTO blue VALUES ('Wang', 'Daisy');
INSERT INTO blue VALUES ('Wang', 'Xin');

INSERT INTO red VALUES ('Wang', 'Daisy');
INSERT INTO red VALUES ('Wang', 'Xin');
INSERT INTO red VALUES ('Wang', 'Xin');

SELECT * FROM blue;

In [ ]:
%sql SELECT * FROM red;

## Representing multiset relations as counted-set relations

### Use a CTAS statement with group by to convert standard tables to counted-set tables

In [ ]:
%%sql
DROP TABLE IF EXISTS bluem;
CREATE TABLE bluem AS 
    SELECT *, COUNT(*) AS multiplicity
    FROM blue
    GROUP BY last, first;

SELECT * FROM bluem;

In [ ]:
%%sql
DROP TABLE IF EXISTS redm;
CREATE TABLE redm AS
    SELECT *, COUNT(*) AS multiplicity
    FROM red
    GROUP BY last, first;

SELECT * FROM redm;

### How do we make selection on counted-set tables work like multisets?

This works exactly the same in both cases. There's nothing special here. Applying `WHERE` filters on a counted-set will always yield a set back, because you're only removing rows from a set. By definition, this cannot create an entity that is not a set. 

In [ ]:
%%sql
-- sigma on multiset
SELECT * FROM blue 
WHERE first = 'Daisy';

In [ ]:
%%sql
-- sigma on counted set
SELECT * FROM bluem WHERE first = 'Daisy';

### What about projection?

We might want to be a bit careful here. See, what defines a set uniquely is its key, and in this case, the key is the combination of `(last, first)`. Simply having `last` or just having `first` is not enough to uniquely identify a row.

In [ ]:
%%sql
-- pi on multiset
SELECT last FROM blue;

In fact, you can see that if you simply selected `last` from a counted-set, you'd get a multi-set as your output.

In [ ]:
%%sql
SELECT last FROM bluem;

To convert this to a counted-set again, you need to sum up the multiplicities of the tuples that the `last` field came from.

In [ ]:
%%sql
-- pi on counted set
SELECT last, SUM(multiplicity)
FROM bluem group by last;

### What about cross-product?

In [ ]:
%%sql
-- x on multiset
SELECT * FROM blue, red;

Next, convert *the output* of a multiset cross-product to a counted set as we did before. This is our desired result:

In [ ]:
%%sql
-- convert multiset x to counted set
WITH cte(blast, bfirst, rlast, rfirst)
    AS (SELECT * FROM blue, red)
SELECT *, COUNT(*)
  from cte
GROUP BY blast, bfirst, rlast, rfirst;

Now, what went on in the arithmetic here? We can think this through by pushing the arithmetic into the query!

First, what do you get with naive cross-product of counted sets? You get the names from each table, along with the number of times that each name showed up in its respective table. So, for example, `('Wang', 'Xin')` showed up once in `blue` and twice in `red`.

In [ ]:
%%sql
SELECT * FROM bluem, redm;

What does each row tell us individually? Each row tells us the number of times that the name from the left must be matched with the name from the right in the original cross product between `blue` and `red`. So if you multiply the multiplicities together, you'll get the number of instances of each ordered pair of names in the final cross product

In [ ]:
%%sql
-- fix multiplicity per row
SELECT b.last, b.first, r.last, r.first, b.multiplicity*r.multiplicity
FROM bluem b, redm r;

If we simply wanted to drop duplicates instead of monitoring how many there were (the point of a counted-set), our life would have been a lot easier...

In [ ]:
%%sql
SELECT DISTINCT b.last, b.first, r.last, r.first
FROM blue b, red r;

# [Scratch] Transposing Demo Code

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/postgres
import pandas as pd

In [ ]:
%%sql
DROP TABLE IF EXISTS example;
CREATE TABLE example(name text, age integer, gpa float);
INSERT INTO example VALUES
       ('Patty Perfect', 22, 4.0),
       ('Sameer Soclose', 20, 3.99),
       ('Jacob Excellent', 21, 3.93);

In [ ]:
df = %sql SELECT * FROM example;
df = df.DataFrame()

In [ ]:
display(df)
display(df.dtypes)

In [ ]:
dft = df.transpose()
display(dft)
display(dft.dtypes)

In [ ]:
dftt = df.transpose().transpose()
display(dftt)
display(dftt.dtypes)

In [ ]:
dftt['age'] = dftt['age'].astype(int)
dftt['gpa'] = dftt['gpa'].astype(float)

In [ ]:
dftt.dtypes

In [ ]:
mat = np.array([[1, 0, 0, 1, 20000],
          [0, 1, 0, 2, 10011],
          [0, 0, 1, 3, 50000],
          [0, 0, 1, 3, 10000]])
mat

In [ ]:
df = pd.DataFrame({'CompanyName': ['VW', 'Acura', 'Hona', 'Honda'],
              'Categorical Value': [1, 2, 3, 3],
              'Price': [20000, 10011, 50000, 10000]})
df

In [ ]:
df.dtypes

In [ ]:
# %sql --persist df